<img src=" " height="300" width="300"> 

# <center> R для тервера и матстата. <br>  <br> 6.1 Проверка гипотез. Часть первая. </center>

Данный ноутбук является конспектом по курсу «R для теории вероятностей и математической статистики» (РАНХиГС, 2017-2018). Автор ноутбука [вот этот парень по имени Филипп.](https://vk.com/ppilif) Если у вас для него есть деньги, слава или женщины, он от этого всего не откажется. Ноутбук распространяется на условиях лицензии [Creative Commons Attribution-Share Alike 4.0.](https://creativecommons.org/licenses/by-sa/4.0/) При использовании обязательно упоминание автора курса и аффилиации. При наличии технической возможности необходимо также указать активную гиперссылку на [страницу курса.](https://fulyankin.github.io/R_probability/) На ней можно найти другие материалы. Фрагменты кода, включенные в этот notebook, публикуются как [общественное достояние.](https://creativecommons.org/publicdomain/zero/1.0/)

---------

Ура! Мы добрались до самого главного. До проверки гипотез. Аналитики в разных компаниях проверяют их на ежедневной основе. Давайте попробуем разобраться зачем. Для начала немного кода для красивых картинок в ggplot2. Как обычно, вы не обязаны понимать этот код. Он тут нужен для красоты. 